In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  1 12:46:56 2023.

@author: souchaud
"""
# %%
import os
import time
import math
import pandas as pd
import numpy as np
from skimage import io
from PIL import Image
from scipy.stats import norm
from scipy import stats
from scipy.signal import find_peaks
from scipy.optimize import curve_fit, minimize_scalar
import imageio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
import matplotlib.gridspec as gridspec
from cycler import cycler  # Importer cycler pour personnaliser le cycle des couleurs
import trackpy as tp
import functions_analyze as lib
import warnings
import importlib
from collections import defaultdict
from colorama import init, Fore, Style
from typing import List, Optional, Union, Any, Dict, Tuple
importlib.reload(lib)

init(autoreset=True)
# from matplotlib.cm import ScalarMappable
# import pdb; pdb.set_trace()
# warnings.simplefilter("always")  # This will always display warnings
# warnings.simplefilter('error', RuntimeWarning)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)

# plt.style.use('dark_background')
plt.style.use('default')  # Réinitialise le style par défaut de Matplotlib pour éviter des conflits avec d'autres styles prédéfinis ou personnalisés.

plt.rcParams.update({  # Mise à jour des paramètres globaux de Matplotlib.

    "figure.facecolor": (0.12, 0.12, 0.12, 1),  # Définit le fond de la figure (zone autour des axes) en gris très foncé (valeurs RGBA).
    "axes.facecolor": (0.12, 0.12, 0.12, 1),  # Définit le fond des axes (zone où les données sont tracées) en gris très foncé.

    "axes.edgecolor": "white",  # Définit la couleur des bordures des axes (rectangle autour de la zone de tracé) en blanc.
    "axes.labelcolor": "white",  # Définit la couleur des étiquettes des axes (titres des axes X et Y) en blanc.

    "xtick.color": "white",  # Définit la couleur des graduations (ticks) sur l'axe X en blanc.
    "ytick.color": "white",  # Définit la couleur des graduations (ticks) sur l'axe Y en blanc.

    "grid.color": "gray",  # Définit la couleur des lignes de la grille (si activée) en gris clair.
    "text.color": "white",  # Définit la couleur par défaut de tous les textes (titres, annotations, etc.) en blanc.

    "axes.titlesize": 16,  # Définit la taille de la police des titres des axes et graphiques.
    "axes.titleweight": "bold",  # Définit le poids de la police des titres (ici, gras).

    "axes.labelsize": 14,  # Définit la taille de la police des étiquettes des axes.
    "axes.labelweight": "medium",  # Définit le poids de la police des étiquettes des axes (ni trop fin ni trop épais).

    "legend.fontsize": 12,  # Définit la taille de la police utilisée pour les légendes des graphiques.

    "lines.linewidth": 2,  # Définit l'épaisseur par défaut des lignes tracées dans les graphiques.
    "lines.markersize": 8,  # Définit la taille par défaut des marqueurs (points, cercles, etc.).

    "font.size": 12,  # Définit la taille de police par défaut pour tous les textes des graphiques.
    "font.family": "sans-serif",  # Définit la famille de polices utilisée par défaut (ici, sans empattement).

    "font.sans-serif": ["Arial", "Helvetica", "DejaVu Sans"],  # Définit une liste de polices sans empattement pour la priorité de rendu.

     "axes.prop_cycle": cycler(color=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]),
 
    # Définit un cycle de couleurs personnalisé pour les tracés multiples sur un même graphique :
    # - Bleu : #1f77b4
    # - Orange : #ff7f0e
    # - Vert : #2ca02c
    # - Rouge : #d62728
})
# set initial time
INITIAL_TIME = time.time()

# experiment parameters
TIME_FRAME = 15 # 15  # 75
SIZE_PIX = 0.637# 1.2773  # 1.634  # 4.902
FPS = 1/TIME_FRAME

# File to study
file_name = 'filtered_final' # 'features'  # 'filtered'Any
# number of frame kept
N_FRAME = 2
N_FRAME_MIN_STUDY = 200  # Nombre minimal de frame sur laquelle la cellule doit être suivi

# nber hours of stydy:
LONG_TIME = False

# Study parameters
ROLLING_MEAN = False
PIXELISATION = False
TIME_FRAME_STUDY = False
DRIFT = False

# plot parameters
IMG_TYPE = 'svg'
ALPHA = 0.5
LINEWIDTH = 0.1
COLOR_SUP = 'blue'
COLOR_INF = 'red'
color_sup_inf = (COLOR_SUP, COLOR_INF)

# % de présences de la particules sur le total de frame étudiées

FRAME_PARTICULE = 1

# ##########
# % de présences des courbes dans les frames

FRAME_PARTICULE = 0.8
# GENERAL PATH 
GENERAL_PATH = '/Users/souchaud/Desktop/Analyses/'
GENERAL_PATH_PICTURES = '/Users/souchaud/Desktop/A_analyser/'
# GENERAL_PATH_PICTURES = '/Volumes/Labo_Alex_Mac/A_analyser/'
############# CONDITION SIMPLE ################
# CONDITION_simple = 'CytoOne_SorC'
# CONDITION_simple = 'NonT_SorC'
CONDITION_simple = 'CytoOne_HL5_10x'
# CONDITION_simple = 'CytoOne_HL5'

############### CONDITION ################
CONDITION = f'{CONDITION_simple}_results_tracking' # _longtime_new_param'
# CONDITION = 'ASMOT035_fiji'
###########################################
# list of pathway to the experiments
PATHWAY_EXPERIMENT = []

if len(PATHWAY_EXPERIMENT) == 0:
    PATHWAY_EXPERIMENT = [f for f in os.listdir(GENERAL_PATH + CONDITION) if
                          os.path.isdir(os.path.join(GENERAL_PATH + CONDITION, f))]
# ##########################   Path Exp final  ###############################

PATHWAY_EXPERIMENT = [f'{GENERAL_PATH}{CONDITION}/' +
                      elem + '/mosaic/' for elem in PATHWAY_EXPERIMENT]

# ##########################   Path to Save pic  ##############################

path_save_pic = f'{GENERAL_PATH}résultats_{CONDITION}_ALL_OK_x5_15s/'

# création d'un dossier spécific d'enregistrement.
if not os.path.exists(path_save_pic):
    os.mkdir(path_save_pic)
os.chdir(path_save_pic)
# read HDF5
importlib.reload(lib)
DATA = lib.read_hdf5_all(pathway_experiment=PATHWAY_EXPERIMENT, name_file=file_name,
                         nbr_frame_min=N_FRAME_MIN_STUDY, condition=CONDITION, drift=DRIFT,
                         search_range=20, memory=5)
# Trier le DataFrame par la colonne "frame"
DATA = DATA.sort_values(by='frame')
# Nombre de particules étudiées au total (avant et après les filtres)
print("Nombre de particule avant tri: ", DATA['particle'].nunique())
DATA = DATA[DATA['frame'] < 340]

filter_data = DATA.groupby('particle').filter(lambda x: len(x) >= N_FRAME_MIN_STUDY)
print("Nombre de particule après le premier tri sur le temps suivi des cellules: ", filter_data['particle'].nunique())
DATA = filter_data
# %% [Compute the DATAS according to some parameters]
if ROLLING_MEAN:
    DATA = lib.rolling_mean(datas=DATA, roll=3)
if PIXELISATION:
    DATA = lib.pixelisation(datas=DATA, size_pix=SIZE_PIX)
if TIME_FRAME_STUDY:
    DATA, TIME_FRAME = lib.keep_nth_image(traj=DATA, n=N_FRAME, time_frame=TIME_FRAME)
# Calcul des vitesses instantanées
DATA['time (min)'] = DATA['frame']*TIME_FRAME/60
DATA = lib.vit_instant_new(traj=DATA, lag_time=TIME_FRAME, pix_size=SIZE_PIX, triage=1)
manips = DATA['experiment'].unique()
num_manips = len(manips)

# Créer une figure pour accueillir tous les subplots
fig = plt.figure(figsize=(16, 6*num_manips))

# Créer un gridspec pour mieux organiser les subplots
gs = gridspec.GridSpec(num_manips, 3, fig)

colors = ['skyblue', 'lightgreen', 'salmon']  # Couleurs pour les différents types de graphiques

for i, manip in enumerate(manips):
    data_manip = DATA[DATA['experiment'] == manip]
    mass_means = data_manip.groupby('particle')['mass'].mean()
    size_means = data_manip.groupby('particle')['size'].mean()

    # Masse moyenne
    ax1 = fig.add_subplot(gs[i, 0])
    ax1.hist(mass_means, bins=100, color=colors[0], density=True)
    ax1.set_title(f"Mean mass of particles for {manip}")
    ax1.set_xlabel("Mean mass")
    ax1.set_ylabel("Density")

    # Taille moyenne
    ax2 = fig.add_subplot(gs[i, 1])
    ax2.hist(size_means, bins=100, color=colors[1], density=True)
    ax2.set_title(f"Mean size of particles for {manip}")
    ax2.set_xlabel("Mean size")
    ax2.set_ylabel("Density")

    # Mass vs. Size au frame 0
    filtered_data = data_manip[data_manip['frame'] == 0]
    ax3 = fig.add_subplot(gs[i, 2])
    
    # Tracer manuellement le graphique Mass vs. Size
    ax3.scatter(
        filtered_data['size'], 
        filtered_data['mass'], 
        c="#d62728",  # Couleur rouge
        edgecolors="#d62728",  # Bordures rouges
        alpha=0.7  # Transparence légère
    )
    ax3.set_title(f"Mass vs. Size at frame 0 for {manip}")
    ax3.set_xlabel("Size")
    ax3.set_ylabel("Mass")
    ax3.grid(True, linestyle="--", alpha=0.5)  # Ajouter une grille pour une meilleure lisibilité

# Ajuster l'espacement entre les subplots
plt.tight_layout()

# Afficher la figure
plt.show()

# Sauvegarder la figure
name = "Mean_Mass_Size_manip"
fig.savefig(f"{path_save_pic}{name}." + IMG_TYPE, format=IMG_TYPE)
# # In[Filter on mass]
# if 'mass' in DATA.columns:
#     mask = DATA.groupby('particle')['mass'].transform('mean') >= 1000
#     DATA = DATA[mask]
# if 'level_0' in DATA.columns:
#     DATA = DATA.drop('level_0', axis=1)
# DATA.reset_index(inplace=True)
# # In[Compute some datas as instant displacement /speed / centering trajectories ...]

# Calculate total path for the n first frames (100)
path_data = lib.calculate_total_path_first_frames(DATA, first_n_frames=100)
# Grouper les données par 'experiment'
grouped = path_data.groupby('experiment')

# Calculer le nombre d'experiments pour déterminer le nombre de subplots nécessaires
n_experiments = len(grouped)

# Créer une figure et des axes pour les subplots
fig, axes = plt.subplots(nrows=n_experiments, figsize=(10, 4*n_experiments))

# Assurer que 'axes' est un array, même s'il n'y a qu'un seul subplot
if n_experiments == 1:
    axes = [axes]

for (experiment, group), ax in zip(grouped, axes):
    # Créer un histogramme pour chaque 'experiment'
    ax.hist(group['total_path_first_n'], bins=10, range=[0, 100], density=True, alpha=0.5)
    ax.set_title(f"Total path in first 100 frames for {experiment}")
    ax.set_xlabel('Lenght path (um)')
    ax.set_ylabel('Density')

# Ajuster l'espacement entre les subplots pour éviter le chevauchement
plt.tight_layout()

# Afficher la figure
plt.show()

# Nombre de cellules avec un déplacement total inférieur à 10 sur les 20 premières frames
num_cells_low_displacement = path_data[path_data['total_path_first_n'] < 15]['particle'].nunique()

print("Nombre de cellules dont le déplacement est trop faible : " , num_cells_low_displacement)
#Reset Index
# # Get the indices of rows to drop
# to_drop = find_swaps_with_return(DATA)

# print(len(to_drop), " movements to delete there while the run is too much.")

# # # Drop the rows from DATA
# DATA = DATA.drop(to_drop)
DATA.reset_index(inplace=True)
# DATA = DATA[DATA['displacement [pix]'] < 5]
DATA = lib.center(traj=DATA)

print("\n"*2)
print(f"Le temps de lecture et de préparation des données pour la condition {CONDITION} est : ",
      (time.time() - INITIAL_TIME), 'sec')
print("\n"*2)
# [Calculation of total and cumulative displacement]
# #############################################################################
DATA, start_end = lib.length_displacement(traj=DATA, size_pix=SIZE_PIX)
# MSD et cutoff
DATA2 = DATA.copy()
DATA2['frame'] = pd.factorize(DATA2['frame'])[0]
IMSD = tp.imsd(traj=DATA2,
               mpp=SIZE_PIX, fps=FPS,
               max_lagtime=200, statistic='msd',
               pos_columns=None)
# %% [traj clustering with fit and defining a cutoff]
LAG_TIME_FIT = 5
# Compute et plot the director factor of the imsd
importlib.reload(lib)

COEF_INF, COEF_SUP, PART_COEF_INF, PART_COEF_SUP, CUTOFF =\
    lib.traj_clustering_with_fit_cutoff(DATA2, imsd=IMSD, hist=True,
                                        lag_time_fit=LAG_TIME_FIT,
                                        micronperpixel=SIZE_PIX,
                                        fps=FPS, binsize=250,
                                        peak_height=50, peak_width=1,
                                        save=True, pathway_fig=path_save_pic,
                                        name='all the experiment autocorr', img_type=IMG_TYPE,
                                        plot=True, color_sup_inf=color_sup_inf,
                                        cutoff_default=0.30
                                        )

# DATA_INF, DATA_SUP, IMSD_INF, IMSD_SUP,
# DATA_INF = DATA[DATA['particle'].isin(PART_COEF_INF)]
# DATA_SUP = DATA[DATA['particle'].isin(PART_COEF_SUP)]
# IMSD_INF = IMSD.loc[:, IMSD.columns.isin(PART_COEF_INF)]
# IMSD_SUP = IMSD.loc[:, IMSD.columns.isin(PART_COEF_SUP)]
# We keep only cell among the cutoff and del all variable useless
DATA = DATA[DATA['particle'].isin(PART_COEF_SUP)]
del DATA2, IMSD, COEF_INF, COEF_SUP, PART_COEF_INF, PART_COEF_SUP, CUTOFF
# Nombre de particules en fonction du temps pour chaque expe
# Obtenir les expériences uniques pour les itérations
experiments = DATA['experiment'].unique()

# Déterminer le nombre de lignes et de colonnes pour les sous-graphiques
# Vous pouvez ajuster cela en fonction du nombre total d'expériences
n_cols = 2  # Nombre de colonnes, ajustez selon le besoin
n_rows = (len(experiments) + n_cols - 1) // n_cols  # Nombre de lignes

# Créer une figure et des axes pour les sous-graphiques
fig, axs = plt.subplots(n_rows, n_cols, figsize=(20 * n_cols, 10 * n_rows))
axs = axs.flatten()  # Aplatir le tableau d'axes si nécessaire

# Tracer les graphiques pour chaque expérience
for i, exp in enumerate(experiments):
    # Grouper les données par 'frame' et calculer le nombre de particules par frame
    nbr_part_per_frame = DATA[DATA['experiment'] == exp].groupby('time (min)')['particle'].nunique()

    # Tracer le graphique sur le sous-graphique correspondant
    ax = axs[i]
    ax.plot(nbr_part_per_frame.index, nbr_part_per_frame.values)
    ax.set_ylim([0, nbr_part_per_frame.max() + 10])
    ax.set_title(f'Nbr particle per Frame - {exp}')
    ax.set_xlabel('time (min)')
    ax.set_ylabel('Number of particle')
    # ax.set_xlim([0, 340])
    # ax.set_ylim([0, 2000])  # Ajustez selon vos données

# Masquer les axes non utilisés s'il y en a
for ax in axs[len(experiments):]:
    ax.axis('off')

# Ajustement de la mise en page
plt.tight_layout()
# Ajustement de l'espacement et des marges
# plt.subplots_adjust(hspace=0.4, wspace=0.4)
# Enregistrer la figure entière
plt.savefig(f"{path_save_pic}Nbr_particle_per_Frame_manip_par_manip." + IMG_TYPE, format=IMG_TYPE)

# Afficher la figure
plt.show()

# %% [Number of particle on each frame]

# Grouper les données par 'frame' et calculer la moyenne de 'VitInst [um/min]'
nbr_part_per_frame = DATA.groupby('time (min)')['particle'].nunique()

lib.plot_datas(x_values=nbr_part_per_frame.index, y_values=nbr_part_per_frame.values,
               title='Nbr particles per Frame',
               x_label='time (min)', y_label='Number of particle',
               x_lim=[0, max(nbr_part_per_frame.index)], y_lim=[0, 10000], save=True,
               path_save_pic=path_save_pic, img_type=IMG_TYPE)
fig, axis = plt.subplots(figsize=(12, 12), dpi=300)  # Augmente la taille et la résolution de la figure
# Assurer une échelle égale pour les axes
axis.set_aspect('equal', 'box')

# Titre du graphique
plt.title('Trajectories after removing suspicious particles', fontsize=16)

# Tracer les trajectoires
tp.plot_traj(DATA, label=False, ax=axis)

# Personnaliser les lignes dans l'axe après leur création
for line in axis.get_lines():  # Accéder à toutes les lignes tracées dans l'axe
    line.set_linewidth(0.1)  # Réduire l'épaisseur des lignes

# Afficher le graphique
plt.show()

# Sauvegarder l'image avec une résolution élevée
fig.savefig(
    path_save_pic + 'Trajectories_after_removing_suspicious_particles.' + IMG_TYPE,
    format=IMG_TYPE,
    dpi=300,  # Haute résolution pour l'image sauvegardée
    bbox_inches='tight'  # Supprime les marges inutiles autour du graphique
)
# Construire le chemin complet pour récupérer les images
image_path = os.path.join(GENERAL_PATH_PICTURES, CONDITION_simple)
image_path = image_path + '_faits'
print(image_path)
# DATA l'ensemble des données que l'on rassemble par expériences
plot_exp = DATA.groupby('experiment')

# Déterminer le nombre de sous-graphiques basé sur le nombre d'expériences
num_experiments = len(plot_exp)
num_cols = 2  # Par exemple, vous pouvez définir 2 colonnes pour vos sous-graphiques
num_rows = (num_experiments + num_cols - 1) // num_cols  # Calculer le nombre de lignes nécessaire

# Créer la figure et les axes pour les sous-graphiques
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, num_rows * 10))  # Ajustez la taille selon vos besoins
axes = axes.flatten()  # Aplatir le tableau d'axes pour une itération facile

for ax, (exp_name, exp_data) in zip(axes, plot_exp):
    exp_directories = []
    for dirpath, dirnames, filenames in os.walk(image_path):
        for dirname in dirnames:
            if exp_name in dirname:
                full_path = os.path.join(dirpath, dirname)
                exp_directories.append(full_path)
    if exp_directories:
        image_path_directory = f'{exp_directories[0]}/mosaic/mosaic_total_0.tif'
        frame = imageio.imread(image_path_directory)
        ax.set_aspect('equal', 'box')
        ax.set_title(f'Trajectories after suspicious particles for {exp_name}')
        tp.plot_traj(exp_data, superimpose=frame, label=False, ax=ax)
    else:
        print(f"No directory found for {exp_name}")
        ax.axis('off')
# Ajuster la mise en page pour éviter le chevauchement
plt.tight_layout()
plt.show()
fig.savefig(path_save_pic + 'trajectories_on_frame_all_experiment.pdf', format='pdf')

series_y = DATA.groupby('experiment')['y'].max().div(2048).apply(math.ceil).astype(int)
series_x = DATA.groupby('experiment')['x'].max().div(2048).apply(math.ceil).astype(int)
# Multiplication élément par élément entre les deux séries
result = series_y * series_x
# %% [Mean speed]
# Grouper les données par 'frame' et calculer la moyenne de 'VitInst [um/min]'
mean_VitInst_per_frame = DATA.groupby('time (min)')['VitInst [um/min]'].mean()
mean_VitInst_per_frame.index = mean_VitInst_per_frame.index

mean_VitInst_per_frame = mean_VitInst_per_frame.rolling(5).mean().dropna()

lib.plot_datas(x_values=mean_VitInst_per_frame.index,
               y_values=mean_VitInst_per_frame.values,
               title='Mean VitInst [um/min] per Frame',
               x_label='time (min)', y_label='Mean VitInst [um/min]',
               x_lim=[0, max(mean_VitInst_per_frame.index)], y_lim=[0, 10], save=True,
               path_save_pic=path_save_pic, img_type="jpg")

DATA2 = DATA.copy()
DATA2.loc[:, 'frame'] = pd.factorize(DATA2['frame'])[0]
IMSD = tp.imsd(traj=DATA2,
               mpp=SIZE_PIX, fps=FPS,
               max_lagtime=200, statistic='msd',
               pos_columns=None)

lib.plot_msd(IMSD, fps=FPS, name="MSD of all frames in function of lag time (s)",
             color_plot = 'red', save=False, pathway_saving=None,
             alpha=0.5, linewidth=0.3, img_type='jpg')
# %% [traj clustering with fit and defining a cutoff]
LAG_TIME_FIT = 5
# Compute et plot the director factor of the imsd
importlib.reload(lib)

COEF_INF, COEF_SUP, PART_COEF_INF, PART_COEF_SUP, CUTOFF =\
    lib.traj_clustering_with_fit_cutoff(DATA2, imsd=IMSD, hist=True,
                                        lag_time_fit=LAG_TIME_FIT,
                                        micronperpixel=SIZE_PIX,
                                        fps=FPS, binsize=250,
                                        peak_height=50, peak_width=1,
                                        save=True, pathway_fig=path_save_pic,
                                        name='all the experiment autocorr', img_type="jpg",
                                        plot=True, color_sup_inf=color_sup_inf,
                                        cutoff_default=1.25
                                        )

# # DATA_INF, DATA_SUP, IMSD_INF, IMSD_SUP,
# DATA_INF = DATA[DATA['particle'].isin(PART_COEF_INF)]
# DATA_SUP = DATA[DATA['particle'].isin(PART_COEF_SUP)]
# IMSD_INF = IMSD.loc[:, IMSD.columns.isin(PART_COEF_INF)]
# IMSD_SUP = IMSD.loc[:, IMSD.columns.isin(PART_COEF_SUP)]
# Appel de la fonction de tracé avec des paramètres stylistiques
lib.plot_displacement(
    DATA, 
    start_end=start_end, 
    alpha=0.5, 
    linewidth=0.5, 
    ylim=[0, 750], 
    xlim=[0, max(DATA['time (min)'])]
)

import matplotlib.pyplot as plt

# Création de la figure avec trois sous-graphiques (normalisés en termes de densité)
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 18), sharex=True, sharey=True)  # Partager les axes x et y pour une comparaison cohérente

# Sous-graphique 1 : Toutes les particules (normalisé en densité)
axes[0].hist(start_end, bins=250, color='royalblue', alpha=0.7, density=True)
axes[0].set_xlim([0, 400])
axes[0].set_title('Histogramme des Distances Début-Fin - Toutes les Particules (Densité)', fontsize=16)
axes[0].set_ylabel('Densité', fontsize=14)
axes[0].grid(True, linestyle='--', alpha=0.6)

# Sous-graphique 2 : Particules Supérieures (normalisé en densité)
axes[1].hist(start_end[start_end.index.isin(PART_COEF_SUP)], bins=250, color='blue', alpha=0.7, density=True)
axes[1].set_xlim([0, 400])
axes[1].set_title('Histogramme des Distances Début-Fin - Particules Supérieures (Densité)', fontsize=16)
axes[1].set_ylabel('Densité', fontsize=14)
axes[1].grid(True, linestyle='--', alpha=0.6)

# Sous-graphique 3 : Particules Inférieures (normalisé en densité)
axes[2].hist(start_end[start_end.index.isin(PART_COEF_INF)], bins=250, color='red', alpha=0.7, density=True)
axes[2].set_xlim([0, 400])
axes[2].set_title('Histogramme des Distances Début-Fin - Particules Inférieures (Densité)', fontsize=16)
axes[2].set_xlabel('Distance Début-Fin (μm)', fontsize=14)
axes[2].set_ylabel('Densité', fontsize=14)
axes[2].grid(True, linestyle='--', alpha=0.6)

# Ajuster l'espacement entre les sous-graphiques pour éviter le chevauchement
plt.tight_layout()

# Afficher la figure
plt.show()

# Initialiser les valeurs de temps d'incubation (conditions_to_values)
conditions_to_values = {
    'ASMOT127': 4.25, 'ASMOT128': 23.58, 'ASMOT130': 29.58, 'ASMOT132': 4.33, 'ASMOT133': 23.12,
    'ASMOT134': 26.12, 'ASMOT135': 29, 'ASMOT136': 31.12, 'ASMOT137': 47.3, 'ASMOT138': 49.75,
    'ASMOT139': 52.25, 'ASMOT140': 0, 'ASMOT141': 4.08, 'ASMOT142': 16.25, 'ASMOT143': 18.67,
    'ASMOT144': 21.08, 'ASMOT145': 23.83, 'ASMOT146': 42.17, 'ASMOT147': 51.17, 'ASMOT148': 47.17,
    'ASMOT149': 70.92, 'ASMOT150': 66.67, 'ASMOT151': 71.17, 'ASMOT152': 76.33, 'ASMOT153': 78.33,
    'ASMOT154': 94.75, 'ASMOT155': 98.42, 'ASMOT156': 100.25, 'ASMOT157': 46.25, 'ASMOT158': 48.08,
    'ASMOT159': 69.58, 'ASMOT160': 20.25, 'ASMOT161': 22.42, 'ASMOT162': 92.33, 'ASMOT163': 0.0,
    'ASMOT164': 5.25, 'ASMOT165': 23.08, 'ASMOT166': 20.08, 'ASMOT167': 21.08, 'ASMOT168': 21.08,
    'ASMOT169': 21.08, 'ASMOT170': 4.58, 'ASMOT171': 7.00, 'ASMOT172': 24.00, 'ASMOT173': 25.58,
    'ASMOT174': 30.58, 'ASMOT175': 47.58, 'ASMOT176': 71.83, 'ASMOT177': 76.67, 'ASMOT178': 77.67,
    'ASMOT179': 95.08, 'ASMOT180': 97.58, 'ASMOT181': 21.08,
}

# Ajouter les valeurs de temps d'incubation au DataFrame
DATA['time to incubation (hours)'] = DATA['experiment'].map(conditions_to_values).fillna(0.0)

# Calcul des sommes de déplacement des particules par expérience et particule
sums_df = (
    DATA.groupby(['experiment', 'particle'])['displacement [pix]']
    .apply(lambda x: x.head(200).sum())
    .reset_index(name='displacement_sum')
)

# Calcul de la médiane des vitesses moyennes par expérience
medians = DATA.groupby('experiment')['VitInst [um/min]'].median()

# Extraction du temps d'incubation et du nombre de particules par expérience
exp_hours = (
    DATA[DATA['time to incubation (hours)'] != 0]
    .groupby('experiment')
    .agg({'time to incubation (hours)': 'first', 'particle': 'nunique'})
    .reset_index()
    .rename(columns={'particle': 'number of particles'})
)

# Calcul des caractéristiques pour chaque expérience
result_df = DATA[DATA['frame'] < 200].groupby('experiment').apply(lambda exp_data: {
    'experiment': exp_data['experiment'].iloc[0],
    'taille': math.ceil(exp_data['y'].max() / 2048) * math.ceil(exp_data['x'].max() / 2048),
    'nombre_part_par_champs': exp_data['particle'].nunique() / (
        math.ceil(exp_data['y'].max() / 2048) * math.ceil(exp_data['x'].max() / 2048)
    ) if (math.ceil(exp_data['y'].max() / 2048) * math.ceil(exp_data['x'].max() / 2048)) > 0 else 0,
    'mean_sum': sums_df[sums_df['experiment'] == exp_data['experiment'].iloc[0]]['displacement_sum'].mean(),
    'mean_speed [um/min]': exp_data.groupby('particle')['VitInst [um/min]'].mean().mean(),
    'time_exp [hours]': conditions_to_values.get(exp_data['experiment'].iloc[0], 0.0),
    'median_speed [um/min]': medians.get(exp_data['experiment'].iloc[0], 0.0)
}).apply(pd.Series).reset_index(drop=True)

# Création de l'histogramme des déplacements moyens par expérience
mean_sum = sums_df.groupby('experiment')['displacement_sum'].mean()
plt.figure(figsize=(10, 6))
plt.hist(mean_sum, bins=25, color='skyblue', alpha=0.7)
plt.title("Nombre d'expériences en fonction du déplacement moyen par particule", fontsize=16)
plt.xlabel("Déplacement moyen (pix)", fontsize=14)
plt.ylabel("Nombre d'expériences", fontsize=14)
plt.grid(True, linestyle='--', alpha=0.6)
plt.savefig(path_save_pic + f"Nbr of exp in function of mean displacement per particule {CONDITION_simple}." + IMG_TYPE, format=IMG_TYPE)
plt.show()

# result_df pour tracer average distance et mean speed en fonction de time to incubation

# Création d'une grille de sous-graphiques 2x2
fig, axes = plt.subplots(1, 2, figsize=(20, 6))  # Ajustez la taille selon vos besoins

# Aplatir le tableau d'axes pour un accès plus facile
ax1, ax2, = axes.flatten()

# Premier graphique: Mean Sum vs. Nombre de Particules par Champ
ax1.scatter(result_df['nombre_part_par_champs'], result_df['mean_sum'], marker='+', color='orange')
ax1.set_title('Average distance traveled vs. Number of particles per field')
ax1.set_xlabel('Number of particles per field')
ax1.set_ylabel('Average distance traveled')

# Deuxième graphique: Mean Speed vs. Nombre de Particules par Champ
ax2.scatter(result_df['nombre_part_par_champs'], result_df['mean_speed [um/min]'], marker='+', color='blue')
ax2.set_title('Mean Speed vs. Number of particles per field')
ax2.set_xlabel('Number of particles per field')
ax2.set_ylabel('Mean Speed [um/min]')

# # Masquer le quatrième axe car il n'est pas utilisé
# ax4.axis('off')

# Ajustement de la mise en page pour éviter le chevauchement des titres
plt.tight_layout()

# Afficher les graphiques
plt.show()

fig.savefig(path_save_pic + f"graph in function of nbr of particles per field {CONDITION_simple}.png", format='png') 
# Création d'une grille de sous-graphiques 2x2
fig, axes = plt.subplots(1, 2, figsize=(20, 6))  # Ajustez la taille selon vos besoins

# Aplatir le tableau d'axes pour un accès plus facile
ax1, ax2, = axes.flatten()

# Premier graphique: Mean Sum vs. Nombre de Particules par Champ
ax1.scatter(result_df['time_exp [hours]'], result_df['mean_sum'], marker='+', color='orange')
ax1.set_title('Average distance traveled vs. time to incubation (hours')
ax1.set_xlabel('time to incubation (hours')
ax1.set_ylabel('Average distance traveled')

# Deuxième graphique: Mean Speed vs. Nombre de Particules par Champ
ax2.scatter(result_df['time_exp [hours]'], result_df['mean_speed [um/min]'], marker='+', color='blue')
ax2.set_title('Mean Speed vs. time to incubation (hours')
ax2.set_xlabel('time to incubation (hours')
ax2.set_ylabel('Mean Speed [um/min]')

# # Masquer le quatrième axe car il n'est pas utilisé
# ax4.axis('off')

# Ajustement de la mise en page pour éviter le chevauchement des titres
plt.tight_layout()

# Afficher les graphiques
plt.show()

fig.savefig(path_save_pic + f"graph in function of nbr of particles per field {CONDITION_simple}.png", format='png') 
def angle_between_directions(row):
    dx1, dy1 = row['dir_x'], row['dir_y']
    dx2, dy2 = row['dir_x_next'], row['dir_y_next']
    angle_initial = np.arctan2(dy1, dx1)
    angle_final = np.arctan2(dy2, dx2)
    angle_change = angle_final - angle_initial
    # Normalisation de l'angle pour qu'il soit entre -pi et pi
    angle_change = (angle_change + np.pi) % (2 * np.pi) - np.pi
    return np.degrees(angle_change)  # Convertir l'angle normalisé en degrés
df_sup = DATA[DATA['particle'].isin(PART_COEF_SUP)].copy()
# Assurez-vous que votre DataFrame est trié par particule et par frame
df_sup.sort_values(by=['particle', 'frame'], inplace=True)

df_sup['dir_x'] = df_sup.groupby('particle')['x'].diff().fillna(0)
df_sup['dir_y'] = df_sup.groupby('particle')['y'].diff().fillna(0)
# Décaler les directions pour obtenir le vecteur direction au temps t+1 pour chaque particule
df_sup['dir_x_next'] = df_sup.groupby('particle')['dir_x'].shift(-1)
df_sup['dir_y_next'] = df_sup.groupby('particle')['dir_y'].shift(-1)

# Appliquer la fonction pour calculer l'angle entre les directions successives
df_sup['angle_change'] = df_sup.apply(angle_between_directions, axis=1)

# Optionnel : Supprimer les lignes où la direction suivante est NaN, ce qui se produit pour la dernière observation de chaque particule
df_sup.dropna(subset=['dir_x_next', 'dir_y_next'], inplace=True)

# Afficher le résultat
# print(df[['frame', 'particle', 'dir_x', 'dir_y', 'dir_x_next', 'dir_y_next', 'angle_change']])

df_inf= DATA[DATA['particle'].isin(PART_COEF_INF)].copy()
# Assurez-vous que votre DataFrame est trié par particule et par frame
df_inf.sort_values(by=['particle', 'frame'], inplace=True)

df_inf['dir_x'] = df_inf.groupby('particle')['x'].diff().fillna(0)
df_inf['dir_y'] = df_inf.groupby('particle')['y'].diff().fillna(0)
# Décaler les directions pour obtenir le vecteur direction au temps t+1 pour chaque particule
df_inf['dir_x_next'] = df_inf.groupby('particle')['dir_x'].shift(-1)
df_inf['dir_y_next'] = df_inf.groupby('particle')['dir_y'].shift(-1)

# Appliquer la fonction pour calculer l'angle entre les directions successives
df_inf['angle_change'] = df_inf.apply(angle_between_directions, axis=1)

# Optionnel : Supprimer les lignes où la direction suivante est NaN, ce qui se produit pour la dernière observation de chaque particule
df_sup.dropna(subset=['dir_x_next', 'dir_y_next'], inplace=True)

# Afficher le résultat
# print(df[['frame', 'particle', 'dir_x', 'dir_y', 'dir_x_next', 'dir_y_next', 'angle_change']])

# Créer une nouvelle figure avec trois sous-graphiques en colonne
fig, axes = plt.subplots(3, 1, figsize=(10, 18), sharex=True)

# Concaténer les deux dataframes pour toutes les particules
df = pd.concat([df_sup, df_inf])

# Histogramme pour toutes les particules
axes[0].hist(df['angle_change'], bins=1000, alpha=0.5, color='green')
axes[0].set_title('Histogramme normalisé des angles entre directions (toutes les particules)')
axes[0].set_xlabel('angle (en degré °)')
axes[0].set_ylabel('Densité')

# Histogramme pour particules sup
axes[1].hist(df_sup['angle_change'], bins=1000, color='blue', alpha=0.5)  # Remplacer `COLOR_SUP` par 'blue'
axes[1].set_title('Histogramme normalisé des angles entre directions pour les particules sup')
axes[1].set_xlabel('angle (en degré °)')
axes[1].set_ylabel('Densité')

# Histogramme pour particules inf
axes[2].hist(df_inf['angle_change'], bins=1000, color='red', alpha=0.5)  # Remplacer `COLOR_INF` par 'red'
axes[2].set_title('Histogramme normalisé des angles entre directions pour les particules inf')
axes[2].set_xlabel('angle (en degré °)')
axes[2].set_ylabel('Densité')

# Ajuster la mise en page pour éviter les chevauchements
plt.tight_layout()

# Afficher la figure
plt.show()

# Calcul des vitesses instantanées moyennes par particule pour chaque expérience
all_mean_vitinst = DATA.groupby(['experiment', 'particle'])['VitInst [um/min]'].mean().reset_index()

# Calcul de la médiane des vitesses moyennes par expérience
medians = all_mean_vitinst.groupby('experiment')['VitInst [um/min]'].median()

# Calcul des limites globales pour une meilleure visualisation
global_min = all_mean_vitinst['VitInst [um/min]'].min()
global_max = all_mean_vitinst['VitInst [um/min]'].max()
delta = (global_max - global_min) * 0.1
global_min -= delta
global_max += delta

# Trier le DataFrame exp_hours selon le temps d'incubation et réindexer
exp_hours = exp_hours.sort_values(by='time to incubation (hours)').reset_index(drop=True)

# Préparation de la figure et des subplots
n_experiments = len(exp_hours)
fig, axes = plt.subplots(nrows=n_experiments, ncols=1, figsize=(10, 3 * n_experiments), sharex=True, sharey=True)

# S'assurer que 'axes' est un tableau, même s'il n'y a qu'un seul subplot
if n_experiments == 1:
    axes = [axes]

# Générer une palette de couleurs
palette = plt.cm.get_cmap('tab20', n_experiments)
colors = [palette(i) for i in range(n_experiments)]

# Itérer sur chaque expérience triée
for idx, ax in enumerate(axes):
    row = exp_hours.iloc[idx]
    exp = row['experiment']
    hour = row['time to incubation (hours)']
    
    # Sélectionner les données pour l'expérience courante
    data_exp = all_mean_vitinst[all_mean_vitinst['experiment'] == exp]
    
    # Tracer l'histogramme sur le subplot correspondant
    ax.hist(data_exp['VitInst [um/min]'], bins=30, alpha=0.3, range=(global_min, global_max), color=colors[idx])
    ax.set_title(f'Vit. Inst. Moyenne [μm/min] - {exp} (Incubation: {hour}h)')
    ax.set_xlabel('VitInst [μm/min]')
    ax.set_ylabel('Nombre de particules')
    
    # Calculer et tracer la médiane en rouge
    median_value = medians.loc[exp]
    ax.axvline(median_value, color='red', linestyle='dashed', linewidth=1)
    ax.text(
        median_value + 0.05 * (global_max - global_min),
        ax.get_ylim()[1] * 0.95,
        f'Médiane: {median_value:.2f}',
        color='red',
        ha='right'
    )
    
    # Appliquer les mêmes limites d'axes à tous les subplots
    ax.set_xlim(global_min, global_max)

# Ajuster l'espacement entre les subplots pour éviter le chevauchement
plt.tight_layout()

# Afficher la figure
plt.show()

# Calcul des vitesses instantanées moyennes par particule pour chaque expérience
all_mean_vitinst_inf = df_inf.groupby(['experiment', 'particle'])['VitInst [um/min]'].mean().reset_index()
all_mean_vitinst_sup = df_sup.groupby(['experiment', 'particle'])['VitInst [um/min]'].mean().reset_index()

# Calcul de la médiane des vitesses moyennes par expérience
medians_inf = all_mean_vitinst_inf.groupby('experiment')['VitInst [um/min]'].median()
medians_sup = all_mean_vitinst_sup.groupby('experiment')['VitInst [um/min]'].median()

# Calcul des limites globales pour une meilleure visualisation
global_min_inf, global_max_inf = all_mean_vitinst_inf['VitInst [um/min]'].min(), all_mean_vitinst_inf['VitInst [um/min]'].max()
global_min_sup, global_max_inf = global_min_inf - (global_max_inf - global_min_inf) * 0.1, global_max_inf + (global_max_inf - global_min_inf) * 0.1
global_min_sup, global_max_sup = all_mean_vitinst_sup['VitInst [um/min]'].min(), all_mean_vitinst_sup['VitInst [um/min]'].max()
global_min_sup, global_max_sup = global_min_sup - (global_max_sup - global_min_sup) * 0.1, global_max_sup + (global_max_sup - global_min_sup) * 0.1

# Préparation de la figure et des subplots
n_experiments = len(exp_hours)
fig, axes = plt.subplots(nrows=n_experiments, ncols=1, figsize=(10, 3 * n_experiments), sharex=True, sharey=True)

if n_experiments == 1:
    axes = [axes]  # S'assurer que 'axes' est itérable

# Générer une palette de couleurs
palette = plt.cm.get_cmap('tab20', n_experiments)

# Itérer sur chaque expérience
for ax, (idx, row) in zip(axes, exp_hours.iterrows()):
    exp = row['experiment']
    hour = row['time to incubation (hours)']
    # Sélectionner les données pour l'expérience courante
    data_exp_inf = all_mean_vitinst_inf[all_mean_vitinst_inf['experiment'] == exp]
    data_exp_sup = all_mean_vitinst_sup[all_mean_vitinst_sup['experiment'] == exp]

    # Tracer l'histogramme sur le subplot correspondant
    ax.hist(data_exp_inf['VitInst [um/min]'], bins=30, alpha=0.3, range=(global_min_inf, global_max_inf), color="red")
    ax.hist(data_exp_sup['VitInst [um/min]'], bins=30, alpha=0.3, range=(global_min_sup, global_max_sup), color="blue")
    ax.set_title(f'Vit. Inst. Moyenne [um/min] - {exp} (Incubation: {hour}h)')
    ax.set_xlabel('VitInst [um/min]')
    ax.set_ylabel('Nombre de particules')
    
    # Calculer et tracer la médiane en rouge
    median_value = medians.loc[exp]
    ax.axvline(median_value, color='red', linestyle='dashed', linewidth=1)
    ax.text(median_value + 0.05 * (global_max - global_min), ax.get_ylim()[1]*0.95, f'Median: {median_value:.2f}', color='red', ha='right')

    # Appliquer les mêmes limites d'axes à tous les subplots
    ax.set_xlim(global_min, global_max)

# Ajuster l'espacement entre les subplots pour éviter le chevauchement
plt.tight_layout()

# Afficher la figure
plt.show()
# Histogramme des vitesses instantanées - toutes les particules
plt.figure(figsize=(12, 12))

# Sous-graphique 1 : Histogramme des vitesses moyennes pour toutes les particules
plt.subplot(2, 1, 1)
plt.hist(DATA.groupby('particle')['VitInst [um/min]'].mean(), bins=500, color='gray', alpha=0.6)
plt.xlabel('Vitesses instantanées moyennes (μm/min)')
plt.ylabel('Densité')
plt.title('Histogramme des vitesses instantanées moyennes (toutes les particules)')

# Sous-graphique 2 : Comparaison des particules supérieures et inférieures
plt.subplot(2, 1, 2)
plt.hist(df_sup.groupby('particle')['VitInst [um/min]'].mean(), bins=500, color='blue', alpha=0.8, label='Particules Supérieures')
plt.hist(df_inf.groupby('particle')['VitInst [um/min]'].mean(), bins=500, color='red', alpha=0.5, label='Particules Inférieures')
plt.xlabel('Vitesses instantanées moyennes (μm/min)')
plt.ylabel('Densité')
plt.title('Histogramme des vitesses instantanées moyennes (Particules Supérieures vs Inférieures)')
plt.legend()

# Ajuster l'espacement entre les sous-graphes
plt.tight_layout()

# Afficher la figure
plt.show()
import matplotlib.pyplot as plt

# Trier result_df par temps d'incubation
result_df_sorted = result_df.sort_values(by='time_exp [hours]')

# Création de la figure pour tracer le graphique
plt.figure(figsize=(10, 5))

# Tracé de la médiane des vitesses instantanées moyennes en fonction du temps d'incubation
plt.scatter(result_df_sorted['time_exp [hours]'], result_df_sorted['median_speed [um/min]'], color='blue', label='Médiane des vitesses instantanées')
plt.plot(result_df_sorted['time_exp [hours]'], result_df_sorted['median_speed [um/min]'], linestyle='--', color='blue', alpha=0.6)

# Ajout de titres et de labels
plt.title('Médiane des vitesses instantanées moyennes en fonction du temps d\'incubation', fontsize=16)
plt.xlabel('Temps d\'incubation (heures)', fontsize=14)
plt.ylabel('Médiane de la vitesse instantanée moyenne [μm/min]', fontsize=14)
plt.ylim([0, 15])

# Ajouter une grille et ajuster le graphique
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()

# Afficher la figure
plt.show()

import matplotlib.pyplot as plt

# Fonction pour préparer les données
def prepare_data(df):
    # Filtrer et trier les expériences par leur temps d'incubation
    incubation_times = df.groupby('experiment')['time to incubation (hours)'].first()
    incubation_times = incubation_times[incubation_times != 0]
    incubation_times = incubation_times.sort_values()

    # Calcul des vitesses instantanées moyennes par particule pour chaque expérience
    all_mean_vitinst = df.groupby(['experiment', 'particle'])['VitInst [um/min]'].median()

    # Calcul de la médiane des vitesses moyennes par expérience
    medians = all_mean_vitinst.groupby(level='experiment').median()

    # Filtrage des médianes pour les expériences valides uniquement
    valid_medians = medians[incubation_times.index]

    return incubation_times, valid_medians

# Préparation des données pour DATA, df_inf, et df_sup
incubation_times_data, valid_medians_data = prepare_data(DATA)
incubation_times_inf, valid_medians_inf = prepare_data(df_inf)
incubation_times_sup, valid_medians_sup = prepare_data(df_sup)

# Création d'une figure pour tracer le graphique
plt.figure(figsize=(12, 6))

# Tracé de la médiane des vitesses instantanées moyennes en fonction du temps d'incubation pour DATA
plt.scatter(incubation_times_data, valid_medians_data, color='red', label='DATA')
plt.plot(incubation_times_data, valid_medians_data, color='red', linestyle='--', alpha=0.6)

# Tracé pour df_inf
plt.scatter(incubation_times_inf, valid_medians_inf, color='green', label='df_inf')
plt.plot(incubation_times_inf, valid_medians_inf, color='green', linestyle='--', alpha=0.6)

# Tracé pour df_sup
plt.scatter(incubation_times_sup, valid_medians_sup, color='blue', label='df_sup')
plt.plot(incubation_times_sup, valid_medians_sup, color='blue', linestyle='--', alpha=0.6)

# Ajout de titres, de labels et de légende
plt.title('Médiane des vitesses instantanées moyennes en fonction du temps d\'incubation', fontsize=16)
plt.xlabel('Temps d\'incubation (heures)', fontsize=14)
plt.ylabel('Médiane de la vitesse instantanée moyenne [μm/min]', fontsize=14)
plt.ylim([0, 10])
plt.legend()

# Ajuster et afficher le graphique
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calcul des nombres uniques de particules pour chaque expérience dans df_inf et df_sup, puis fusion dans un DataFrame unique
unique_particles = pd.concat([
    df_inf.groupby('experiment')['particle'].nunique().rename('inf'),
    df_sup.groupby('experiment')['particle'].nunique().rename('sup')
], axis=1).fillna(0)

# Calcul de la proportion de particules 'inf' et ajout du temps d'incubation
unique_particles['proportion_inf'] = unique_particles['inf'] / (unique_particles['inf'] + unique_particles['sup'])
unique_particles['time_to_incubation'] = df_inf.groupby('experiment')['time to incubation (hours)'].first().reindex(unique_particles.index).fillna(0)
unique_particles = unique_particles[unique_particles['time_to_incubation'] != 0].sort_values('time_to_incubation')

# Création du graphique
fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(unique_particles['time_to_incubation'], unique_particles['proportion_inf'], color='blue', marker='+', s=30, linewidths=0.5)

# Sélection des expériences à annoter
indices_to_annotate = set(unique_particles['proportion_inf'].nlargest(5).index).union(
    unique_particles['proportion_inf'].nsmallest(5).index
).union(unique_particles.index[::max(1, len(unique_particles) // 10)])

# Ajouter des annotations avec décalage pour éviter les chevauchements
for i, experiment in enumerate(unique_particles.index):
    if experiment in indices_to_annotate:
        x, y = unique_particles.loc[experiment, 'time_to_incubation'], unique_particles.loc[experiment, 'proportion_inf']
        if np.isfinite(x) and np.isfinite(y):
            x_offset, y_offset = (0.5 if i % 2 == 0 else -0.5), (0.02 if i % 3 == 0 else -0.02)
            ax.text(x + x_offset, y + y_offset, str(experiment)[-3:], fontsize=8, color='white',
                    ha='center', va='center', bbox=dict(facecolor='black', alpha=0.1, edgecolor='none'))

# Ajout de titres et labels
ax.set_title('Proportion de particules de df_inf par rapport à la somme des particules de df_inf et df_sup')
ax.set_xlabel('Temps d\'incubation (heures)')
ax.set_ylabel('Proportion de particules (df_inf / (df_inf + df_sup))')

# Ajustement des graduations de l'axe x et affichage
ax.set_xticks(np.arange(0, unique_particles['time_to_incubation'].max() + 10, 10))
plt.grid(True)
plt.tight_layout()
plt.show()

# Sauvegarde de la figure
fig.savefig(f"{path_save_pic}Mean_Mass_Size_manip." + IMG_TYPE, format=IMG_TYPE)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Filtrer les expériences avec un temps d'incubation non nul
unique_particles = unique_particles[unique_particles['time_to_incubation'] != 0]

# Création des tranches de temps de 10 heures
bins = np.arange(0, unique_particles['time_to_incubation'].max() + 10, 10)
labels = [f"{int(left)}-{int(right)}" for left, right in zip(bins[:-1], bins[1:])]

# Assignation des expériences à des tranches de temps
unique_particles['time_tranche'] = pd.cut(unique_particles['time_to_incubation'], bins=bins, labels=labels, right=False)

# Groupe par tranche de temps et calcul de la moyenne des proportions pour inf et sup
proportion_per_tranche_inf = unique_particles.groupby('time_tranche')['proportion_inf'].mean()
proportion_per_tranche_sup = 1 - proportion_per_tranche_inf  # La proportion des particules sup est simplement 1 - proportion_inf

# Création d'une figure pour tracer le graphique
fig, ax = plt.subplots(figsize=(12, 6))

# Tracé des moyennes de proportion par tranche de temps pour inf et sup
x = np.arange(len(proportion_per_tranche_inf))
width = 0.35  # Largeur des barres

ax.bar(x - width/2, proportion_per_tranche_inf, width, color='blue', alpha=0.7, label='Proportion Inf')
ax.bar(x + width/2, proportion_per_tranche_sup, width, color='red', alpha=0.7, label='Proportion Sup')

# Ajout de titres et de labels
ax.set_title('Proportion de particules de df_inf et df_sup par tranche de temps d\'incubation (10 heures)')
ax.set_xlabel('Tranche de temps d\'incubation (heures)')
ax.set_ylabel('Proportion moyenne de particules')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45)
ax.legend()

# Ajuster et afficher le graphique
plt.grid(True)
plt.tight_layout()
plt.show()

# Sauvegarder la figure
name = "proportion_inf_sup_vs_time"
fig.savefig(f"{path_save_pic}{name}." + IMG_TYPE, format=IMG_TYPE)

# Appel de la fonction de tracé avec des paramètres stylistiques
lib.plot_displacement_low_and_high(traj_sup=df_sup, traj_inf=df_inf, part_coef_inf=PART_COEF_INF, part_coef_sup=PART_COEF_SUP,
                                   start_end=start_end,
                                   save=True,
                                   pathway_saving=path_save_pic, name="displacement_start-end_time", img_type=IMG_TYPE)